In [1]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [2]:
from fosforml.model_manager.snowflakesession import get_session

In [3]:
session = get_session()
session.get_current_schema().replace('"','')


'INSIGHT_SPCS_SCHEMA'

In [62]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

In [63]:
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [64]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")

In [65]:
# df['SALARY']

In [66]:
# snow_df = session.table("HR_CHURN")
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop(['Unnamed: 0','DEPARTMENT'])
snow_df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |low       |
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |medium    |
|0.11                  |0.88               |7                 |272                     |4                     |0        

In [67]:
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
input_data_frame.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |low       |
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |medium    |
|0.11                  |0.88               |7                 |272                     |4                     |0        

In [69]:
label_encoder_column = []     
# input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"SALARY"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |low       |
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |medium    |
|0.11                  |0.88               |7                 |272                     |4                     |0        

In [70]:
from snowflake.ml.modeling.pipeline import Pipeline

In [85]:
model = RandomForestClassifier(input_cols=feature_cols, label_cols=target_col)

In [82]:
# pipeline = Pipeline(
#     steps = [
#         ("SnowflakePiplineModel",
#          XGBClassifier(input_cols=feature_cols, label_cols=target_col))
#     ]
# )

In [86]:
model.fit(input_data_frame)

In [76]:
# test_en_df = apply_label_encoding(test_df)

In [87]:
pred_snow_df = model.predict(test_df)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [88]:
prob_snow_df = model.predict_proba(test_en_df)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


In [ ]:
test_df = prob_snow_df.to_pandas()

In [ ]:
test_df.head()

In [18]:
test_df['SALARY'].unique()

array([1., 2., 0.])

In [19]:
test_df.columns

Index(['SALARY', 'DEPARTMENT', 'SATISFACTION_LEVEL', 'LAST_EVALUATION',
       'NUMBER_PROJECT', 'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY',
       'WORK_ACCIDENT', 'LEFT', 'PROMOTION_LAST_5YEARS', '"PREDICT_PROBA_0.0"',
       '"PREDICT_PROBA_1.0"', '"PREDICT_PROBA_2.0"'],
      dtype='object')

In [21]:
# dir(pipeline)

In [53]:
df = df.drop(['Unnamed: 0','DEPARTMENT'],axis=1).dropna()

In [61]:
from snowflake.ensemble._forest import RandomForestClassifier
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'snowflake.ensemble'

In [55]:
model = RandomForestClassifier()
x_train,x_test,y_train,y_test = train_test_split(df.drop('SALARY',axis=1),df['SALARY'])

In [56]:
x_train.columns

Index(['SATISFACTION_LEVEL', 'LAST_EVALUATION', 'NUMBER_PROJECT',
       'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY', 'WORK_ACCIDENT', 'LEFT',
       'PROMOTION_LAST_5YEARS'],
      dtype='object')

In [57]:
x_train.head()

SATISFACTION_LEVEL  LAST_EVALUATION  NUMBER_PROJECT  \
2692                 0.68             0.51               4   
13580                0.86             0.84               3   
7448                 0.55             0.98               4   
3409                 0.58             0.86               5   
570                  0.44             0.52               2   

       AVERAGE_MONTLY_HOURS  TIME_SPEND_COMPANY  WORK_ACCIDENT  LEFT  \
2692                    157                   3              0     0   
13580                   177                   3              0     0   
7448                    137                   2              0     0   
3409                    206                   3              0     0   
570                     137                   3              0     1   

       PROMOTION_LAST_5YEARS  
2692                       0  
13580                      0  
7448                       0  
3409                       0  
570                        0

In [58]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [60]:
# model.predict_proba(x_test)

In [46]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="Testmode02092024",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="Notebook",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODE02092024' registered successfully."

In [54]:
model_obj = model_registry.get_model(model_name="TestModel")

In [55]:
model = model_obj.version(version_name="v1")

In [58]:
inptu_df ,test_df = session.table("HR_CHURN").randomSplit([0.75, 0.25])

In [59]:
test_df = apply_label_encoding(test_df)

In [75]:
test_df_pandas = test_df.to_pandas()

In [76]:
session.write_pandas(test_df_pandas,table_name="TEST_DATA_HR",overwrite=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [80]:
test_df_fetch = session.table("TEST_DATA_HR")

In [81]:
test_df_fetch.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [82]:
model.run(test_df_fetch, function_name="predict").to_pandas()

SALARY  DEPARTMENT  Unnamed: 0  SATISFACTION_LEVEL  LAST_EVALUATION  \
0        1.0         3.0          34                0.84             0.87   
1        2.0         6.0          66                0.43             0.54   
2        1.0         7.0          94                0.09             0.83   
3        1.0         8.0         130                0.10             0.83   
4        1.0         7.0         163                0.79             0.89   
...      ...         ...         ...                 ...              ...   
7447     2.0         5.0       14821                0.09             0.95   
7448     2.0         9.0       14859                0.45             0.54   
7449     1.0         7.0       14888                0.39             0.49   
7450     2.0         7.0       14945                0.14             0.75   
7451     2.0         2.0       14974                0.36             0.54   

      NUMBER_PROJECT  AVERAGE_MONTLY_HOURS  TIME_SPEND_COMPANY  WORK_ACCIDENT  \
0                  4                   246                   6              0   
1                  2                   153                   3              0   
2                  6                   255                   4              0   
3                  6                   292                   4              0   
4                  5                   239                   5              0   
...              ...                   ...                 ...            ...   
7447               7                   256                   4              0   
7448               2                   129                   3              0   
7449               2                   142                   3              0   
7450               4                   277                   5              1   
7451               2                   153                   3              0   

      LEFT  PROMOTION_LAST_5YEARS  OUTPUT_SALARY  
0        1                      0            1.0  
1        1                      0            2.0  
2        1                      0            1.0  
3        1                      0            1.0  
4        1                      0            1.0  
...    ...                    ...            ...  
7447     1                      0            2.0  
7448     1                      0            2.0  
7449     1                      0            1.0  
7450     1                      0            1.0  
7451     1                      0            2.0  

[7452 rows x 12 columns]

In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)